<a href="https://colab.research.google.com/github/AdarshSRM/FirstSite/blob/main/Poringa_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- SET YOUR URL HERE ---
TARGET_URL = "https://www.poringa.net/posts/imagenes/2011655/Debora-Natalin-Pistarchi---100-Fotos-Hot-Para-Infierno-Rojo.html"
# -------------------------
print(f"Target URL set to: {TARGET_URL}")

Target URL set to: https://www.poringa.net/posts/imagenes/2011655/Debora-Natalin-Pistarchi---100-Fotos-Hot-Para-Infierno-Rojo.html


In [2]:
import shutil
import os

to_remove = ['downloads', 'imagebam_links.txt', 'result_images.zip']

for item in to_remove:
    if os.path.exists(item):
        if os.path.isdir(item):
            shutil.rmtree(item)
            print(f"Removed folder: {item}")
        else:
            os.remove(item)
            print(f"Removed file: {item}")

print("Cleanup complete. Ready for new extraction.")

Cleanup complete. Ready for new extraction.


In [3]:
import requests
from bs4 import BeautifulSoup

def scrape_links():
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    print(f"Fetching links from: {TARGET_URL}")

    try:
        response = requests.get(TARGET_URL, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract links in order and remove duplicates
        links = [a['href'] for a in soup.find_all('a', href=True) if "imagebam.com/image/" in a['href']]
        unique_links = list(dict.fromkeys(links))

        if unique_links:
            with open("imagebam_links.txt", "w") as f:
                for link in unique_links:
                    f.write(link + "\n")
            print(f"Success! Found {len(unique_links)} links. Saved to 'imagebam_links.txt'.")
        else:
            print("No ImageBam links found.")
    except Exception as e:
        print(f"Error: {e}")

scrape_links()

Fetching links from: https://www.poringa.net/posts/imagenes/2011655/Debora-Natalin-Pistarchi---100-Fotos-Hot-Para-Infierno-Rojo.html
Success! Found 100 links. Saved to 'imagebam_links.txt'.


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

os.makedirs("downloads", exist_ok=True)

def resolve_direct_link(session, url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36", "Referer": TARGET_URL}
    session.get(url, headers=headers)
    headers["Referer"] = url
    response = session.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    img = soup.find('img', id='main-image') or soup.select_one('img[src*="imagebam.com"]')
    return img.get('src') if img else None

def download_images():
    if not os.path.exists("imagebam_links.txt"):
        print("Error: Run Cell 3 first!")
        return

    with open("imagebam_links.txt", "r") as f:
        urls = [line.strip() for line in f.readlines()]

    session = requests.Session()
    session.cookies.set("nsfw_inter", "1", domain=".imagebam.com")

    print(f"Downloading {len(urls)} images...")
    for index, url in enumerate(urls, start=1):
        try:
            direct_url = resolve_direct_link(session, url)
            if direct_url:
                original_id = url.split('/')[-1]
                filename = f"downloads/{index}-{original_id}.jpg"
                img_data = session.get(direct_url, headers={"Referer": url}).content
                with open(filename, 'wb') as f:
                    f.write(img_data)
                print(f"[{index}/{len(urls)}] Saved: {filename}")
            else:
                print(f"[{index}] Could not resolve: {url}")
            time.sleep(1)
        except Exception as e:
            print(f"Error on {url}: {e}")

download_images()

[1/100] Saved: downloads/1-fd704c147366835.jpg
[2/100] Saved: downloads/2-bcaba1147366848.jpg
[3/100] Saved: downloads/3-7492ea147366856.jpg
[4/100] Saved: downloads/4-3a741f147366862.jpg
[5/100] Saved: downloads/5-f96143147366868.jpg
[6/100] Saved: downloads/6-c76e5b147366876.jpg
[7/100] Saved: downloads/7-10f5c4147366883.jpg
[8/100] Saved: downloads/8-04f439147366889.jpg
[9/100] Saved: downloads/9-ba6a34147366901.jpg
[10/100] Saved: downloads/10-93d2ed147366911.jpg
[11/100] Saved: downloads/11-43f625147366920.jpg
[12/100] Saved: downloads/12-44c3d1147366930.jpg


KeyboardInterrupt: 

In [5]:
import os
from google.colab import files

if os.path.exists("downloads") and len(os.listdir("downloads")) > 0:
    print("Zipping files...")
    !zip -q -r result_images.zip downloads
    print("Download starting...")
    files.download('result_images.zip')
else:
    print("Nothing to zip. Make sure the download cell (Cell 4) finished successfully.")

Zipping files...
Download starting...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>